# Digital Trigger capabilities

The digital trigger is a simple digital output, mapped to ADCIO 0 to 7.
For the location pf these pins, refer to the data sheets, schematics or documentation of the concrete board.

- [ZCU216 Platform](https://docs.xilinx.com/r/en-US/ug1390-zcu216-eval-bd/Header)
- [ZCU111 Platform](https://docs.xilinx.com/r/en-US/ug1271-zcu111-eval-bd/XM500-ADC/DAC-Data-and-Clock-SMA)

Digital Triggers have a set of outputs that are activated by one call to `DigitalTrigger` and a length that defines how long these triggers are active.

In [1]:
import qiclib as ql
from qiclib.code import *

platform = "slot6-platform"
qic = ql.QiController(platform)

QKIT configuration initialized -> available as qkit.cfg[...]


/home/lukas/.local/lib/python3.10/site-packages/qiclib/packages/qkit_polyfill.py:38: UserWarning: If you intend to use the QiController together with Qkit, do not forget to start it with qkit.start() before creating the QiController instance.
  warnings.warn(


[QiController] qiclib version: undetectable
[QiController] Establishing remote connection to slot6-platform...
[QiController] Detected QiController running on ZCU216 board
[QiController] Firmware build time: 08.03.2024 09:21:19 (Revision D0B1168B)
[QiController] Firmware with 4 digital unit cells detected.


# Simple triggers

The next cells contain simple triggers that all output a pulse of 20 ns length

In [2]:
with QiJob() as simple_trigger:
    q = QiCells(1)
    DigitalTrigger(q[0], 20e-9, outputs=[0])

simple_trigger.run(qic)

HTML(value="<table style='width:100%'><tr><td> (0/1) </td><td>&#9992; -?-    </td><td>&#128336;  --:--:--   (e…

IntProgress(value=0, description='Averages', layout=Layout(width='95%'), max=1)

## Triggering multiple outputs

This next cell triggers digital outputs 0, 1 and 7.

In [3]:
with QiJob() as multiple_outputs:
    q = QiCells(1)
    DigitalTrigger(q[0], 20e-9, [0, 1, 7])

multiple_outputs.run(qic)

HTML(value="<table style='width:100%'><tr><td> (0/1) </td><td>&#9992; -?-    </td><td>&#128336;  --:--:--   (e…

IntProgress(value=0, description='Averages', layout=Layout(width='95%'), max=1)

## Adding an output delay

Intrinsic output delays cannot be added dynamically (or, in other words, inside a `QiJob`), but have to be configured prior to running a job.
For dynamic delay, add a `Wait` command before issuing the `DigitalTrigger` command.

The next cell configures digital output 1 to have a delay of 30 ns and digital output 0 without delay.
Consequently, the signal at digital output 1 appears 20 ns later.

In [4]:
with QiJob() as delay:
    q = QiCells(1)
    DigitalTrigger(q[0], 60e-9, [0, 1])

qic.cell[0].digital_trigger.set_delay(0, 0e-9)
qic.cell[0].digital_trigger.set_delay(1, 100e-9)
delay.run(qic)

HTML(value="<table style='width:100%'><tr><td> (0/1) </td><td>&#9992; -?-    </td><td>&#128336;  --:--:--   (e…

IntProgress(value=0, description='Averages', layout=Layout(width='95%'), max=1)

## Triggering from multiple unit cells

In the following `QiJob`, Cell 0 triggers output 0 for 10 ns and cell triggers output 1 for 20 ns.
The otuputs start simultaneously.

In [5]:
with QiJob() as multiple_cells:
    q = QiCells(2)
    DigitalTrigger(q[0], 10e-9, [0])
    DigitalTrigger(q[1], 20e-9, [1])

multiple_cells.run(qic)

HTML(value="<table style='width:100%'><tr><td> (0/1) </td><td>&#9992; -?-    </td><td>&#128336;  --:--:--   (e…

IntProgress(value=0, description='Averages', layout=Layout(width='95%'), max=1)

# Raw hardware control

Sometimes, for example for testing, debugging or to test experimental features, it might be helpful to see configuration options without the `DigitalTrigger` wrapper. For this purpose, the `qiclib.hardware.digital_trigger` module can be used.

In [6]:
from qiclib.hardware.digital_trigger import OutputConfig, TriggerSet

trig = qic.cell[0].digital_trigger

# Setting the output level to 'active_high' is not yet supported
# since this will not ork as intended when multiple cells have the active_high
# mode, or there is a mix between cells that are active high and active low.
# Setting this for one cell only and only using one cell will work, however
config = OutputConfig(level="active_low", delay=0.0)
trig._set_output_config(0, config)
trig.set_trigger_set(1, TriggerSet(duration=20e-9, outputs=[1], continuous=False))
trig.trigger(1)